# Keyphrase extraction using BERT
I used almost the same technic than this [paper](https://arxiv.org/pdf/1801.04470).
I use bert to get the pooler of the document and for each sentence.
Then, I rank each phrase depends on this cos similarity to the document's pooler.

Finally, I use the MMR score (used on the article when I cited above)
to remove the  too similar top ranked key-phrases.

# Depedencies

In [1]:
import json
import spacy
import torch
from torch import nn
from transformers import CamembertModel, CamembertTokenizerFast
nlp = spacy.load('fr_core_news_sm')

# Read the HAS document

In [3]:
has_corpus = json.load(open('./processed-HAS.json'))
example = list(has_corpus.items())[60]
example

('https://www.has-sante.fr/jcms/c_1623732/fr/depistage-et-prevention-du-cancer-colorectal',
 'Dans ce Référentiel de pratiques de l’examen périodique de santé des Centres d’Examen de Santé (CES) de l’Assurance Maladie sur le dépistage et la prévention du cancer colorectal, la HAS rappelle les recommandations en ce qui concerne le programme national de dépistage organisé du cancer colorectal, les critères d’inclusion et d’exclusion des hommes et des femmes dans ce dépistage et les modalités de prévention primaire par la modification des habitudes de vie. Des données complémentaires sont apportées sur l’histoire de la maladie, l’épidémiologie, les facteurs de risque, les examens et les traitements, la stratégie diagnostique en fonction des niveaux de risque, la place du médecin de CES dans le dispositif de prévention et dépistage de ce cancer.')

# Load the camembert model

In [4]:
model = 'camembert-base'
tokenizer = CamembertTokenizerFast.from_pretrained(model)
camembert = CamembertModel.from_pretrained(model)

#  Compute the pooler for each documents

In [5]:
tokenized_example = tokenizer(example[1], return_attention_mask=True, return_tensors='pt')
print('example of a tokenized sentences (just show up a slice) : ')
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids'].tolist()[0]))
print('get the pooler of the doc : ')
doc_pooler = camembert(tokenized_example['input_ids'])['pooler_output']
print('size of the pooler : ', doc_pooler.shape)

example of a tokenized sentences (just show up a slice) : 
['<s>', '▁Dans', '▁ce', '▁Ré', 'fér', 'entiel', '▁de', '▁pratiques', '▁de', '▁l', '’', 'examen', '▁périodique', '▁de', '▁santé', '▁des', '▁Centre', 's', '▁d', '’', 'Ex', 'amen', '▁de', '▁Santé', '▁(', 'CES', ')', '▁de', '▁l', '’', 'Assurance', '▁Maladie', '▁sur', '▁le', '▁dépistage', '▁et', '▁la', '▁prévention', '▁du', '▁cancer', '▁color', 'ect', 'al', ',', '▁la', '▁H', 'AS', '▁rappelle', '▁les', '▁recommandations', '▁en', '▁ce', '▁qui', '▁concerne', '▁le', '▁programme', '▁national', '▁de', '▁dépistage', '▁organisé', '▁du', '▁cancer', '▁color', 'ect', 'al', ',', '▁les', '▁critères', '▁d', '’', 'inclusion', '▁et', '▁d', '’', 'exclusion', '▁des', '▁hommes', '▁et', '▁des', '▁femmes', '▁dans', '▁ce', '▁dépistage', '▁et', '▁les', '▁modalités', '▁de', '▁prévention', '▁primaire', '▁par', '▁la', '▁modification', '▁des', '▁habitudes', '▁de', '▁vie', '.', '▁Des', '▁données', '▁complémentaires', '▁sont', '▁apportées', '▁sur', '▁l', '’', '

# Parse the key phrase or lexical windows and compute the pooler for each one
for our purpose, I modify slightly the initial parsing of the way to parse the document.
I am using a lexical windows of 10

In [6]:
window_size = 10
offset = 4
slices = [example[1].split(' ')[i:i+window_size] for i in range(len(example[1].split(' '))-2)
          if i % offset == 0]
print(slices)
bert_slice_matrix = torch.stack([
    camembert(
        tokenizer(' '.join(sl), return_attention_mask=True, return_tensors='pt')['input_ids']
    )['pooler_output']
    for sl in slices
]).squeeze(1)

[['Dans', 'ce', 'Référentiel', 'de', 'pratiques', 'de', 'l’examen', 'périodique', 'de', 'santé'], ['pratiques', 'de', 'l’examen', 'périodique', 'de', 'santé', 'des', 'Centres', 'd’Examen', 'de'], ['de', 'santé', 'des', 'Centres', 'd’Examen', 'de', 'Santé', '(CES)', 'de', 'l’Assurance'], ['d’Examen', 'de', 'Santé', '(CES)', 'de', 'l’Assurance', 'Maladie', 'sur', 'le', 'dépistage'], ['de', 'l’Assurance', 'Maladie', 'sur', 'le', 'dépistage', 'et', 'la', 'prévention', 'du'], ['le', 'dépistage', 'et', 'la', 'prévention', 'du', 'cancer', 'colorectal,', 'la', 'HAS'], ['prévention', 'du', 'cancer', 'colorectal,', 'la', 'HAS', 'rappelle', 'les', 'recommandations', 'en'], ['la', 'HAS', 'rappelle', 'les', 'recommandations', 'en', 'ce', 'qui', 'concerne', 'le'], ['recommandations', 'en', 'ce', 'qui', 'concerne', 'le', 'programme', 'national', 'de', 'dépistage'], ['concerne', 'le', 'programme', 'national', 'de', 'dépistage', 'organisé', 'du', 'cancer', 'colorectal,'], ['de', 'dépistage', 'organisé'

# Ranked each keyphrase

In [7]:
cos_sentence_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
top_ranked_vector = torch.argsort(cos_sentence_sim(bert_slice_matrix, doc_pooler))
top_ranked_slices = [slices[vector] for vector in top_ranked_vector]
top_ranked_slices[:5]

[['hommes',
  'et',
  'des',
  'femmes',
  'dans',
  'ce',
  'dépistage',
  'et',
  'les',
  'modalités'],
 ['médecin',
  'de',
  'CES',
  'dans',
  'le',
  'dispositif',
  'de',
  'prévention',
  'et',
  'dépistage'],
 ['et', 'dépistage', 'de', 'ce', 'cancer.'],
 ['risque,',
  'la',
  'place',
  'du',
  'médecin',
  'de',
  'CES',
  'dans',
  'le',
  'dispositif'],
 ['dans',
  'ce',
  'dépistage',
  'et',
  'les',
  'modalités',
  'de',
  'prévention',
  'primaire',
  'par']]

# Use MMR score to remove duplicate

In [9]:
bert_slice_max = torch.stack([
                    torch.max(cos_sentence_sim(
                                    torch.cat([bert_slice_matrix[:idx_slice],
                                               bert_slice_matrix[idx_slice+1:]]),
                        _slice.unsqueeze(0)))
    for idx_slice, _slice in enumerate(bert_slice_matrix)])
lambda_scalar = 0.7
mmr_score = torch.argsort((lambda_scalar*cos_sentence_sim(bert_slice_matrix, doc_pooler)) -
                          (1-lambda_scalar)*bert_slice_max)

mmr_slices = [slices[score] for score in mmr_score]
mmr_slices

[['hommes',
  'et',
  'des',
  'femmes',
  'dans',
  'ce',
  'dépistage',
  'et',
  'les',
  'modalités'],
 ['et', 'dépistage', 'de', 'ce', 'cancer.'],
 ['médecin',
  'de',
  'CES',
  'dans',
  'le',
  'dispositif',
  'de',
  'prévention',
  'et',
  'dépistage'],
 ['risque,',
  'la',
  'place',
  'du',
  'médecin',
  'de',
  'CES',
  'dans',
  'le',
  'dispositif'],
 ['dans',
  'ce',
  'dépistage',
  'et',
  'les',
  'modalités',
  'de',
  'prévention',
  'primaire',
  'par'],
 ['le',
  'dispositif',
  'de',
  'prévention',
  'et',
  'dépistage',
  'de',
  'ce',
  'cancer.'],
 ['la',
  'stratégie',
  'diagnostique',
  'en',
  'fonction',
  'des',
  'niveaux',
  'de',
  'risque,',
  'la'],
 ['primaire',
  'par',
  'la',
  'modification',
  'des',
  'habitudes',
  'de',
  'vie.',
  'Des',
  'données'],
 ['facteurs',
  'de',
  'risque,',
  'les',
  'examens',
  'et',
  'les',
  'traitements,',
  'la',
  'stratégie'],
 ['d’inclusion',
  'et',
  'd’exclusion',
  'des',
  'hommes',
  'et',
 

# Clean the keyphrase and try to extract the more frequent ngram

In [11]:
mmr_str = ' '.join([' '.join(mmr_slice) for mmr_slice in mmr_slices[:5]])
keywords = []
doc = nlp(mmr_str)
for token in doc:
    if not token.is_stop and not token.is_space and not token.is_punct:
        keywords.append(token.lemma_)

set(keywords)

{'cancer',
 'dispositif',
 'dépistage',
 'femme',
 'homme',
 'modalité',
 'médecin',
 'place',
 'primaire',
 'prévention',
 'risqu'}

# write the result to a json and csv file